<a href="https://colab.research.google.com/github/rana6-12/Final-Project-ITI/blob/main/Final_Project_ITI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ٌRana Yasser Anter**

**Upload Data Set**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Replace with the actual path to your file in Google Drive
file_path = '/content/drive/MyDrive/IMDB Dataset.csv'

try:
  df = pd.read_csv(file_path)
  display(df.head())
except FileNotFoundError:
  print(f"Error: The file '{file_path}' was not found. Please check the path and try again.")
except Exception as e:
  print(f"An error occurred: {e}")

Error: The file '/content/drive/MyDrive/IMDB Dataset.csv' was not found. Please check the path and try again.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df.shape

NameError: name 'df' is not defined

In [ ]:
df.head(5)

**# Check columns, types, missing**

In [ ]:
print(df.columns)
print(df.dtypes)
print(df.isna().sum())

**# Class balance**

In [ ]:
print(df['sentiment'].value_counts())
sns.countplot(x='sentiment', data=df)
plt.title("Class distribution")
plt.show()

**# Length features**

In [ ]:
df['n_chars'] = df['review'].astype(str).str.len()
df['n_tokens'] = df['review'].astype(str).str.split().str.len()
print(df[['n_chars','n_tokens']].describe())

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(df['n_tokens'], bins=100, kde=True)
plt.title("Distribution of review token counts")
plt.show()

**Preprocessing :generic **

In [ ]:
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**
# Download NLTK data once**

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

URL_RE = re.compile(r'https?://\S+|www\.\S+')
HTML_RE = re.compile(r'<.*?>')

In [ ]:
def generic_preprocess(text):
    # 1. lowercase
    t = str(text).lower()
    # 2. remove urls & html tags
    t = URL_RE.sub(' ', t)
    t = HTML_RE.sub(' ', t)
    # 3. remove punctuation
    t = t.translate(str.maketrans('', '', string.punctuation))
    # 4. keep letters only (optional) -> here we allow spaces and letters
    t = re.sub(r'[^a-z\s]', ' ', t)
    # 5. collapse spaces
    t = re.sub(r'\s+', ' ', t).strip()
    # 6. tokenize + remove stopwords + lemmatize
    tokens = [w for w in t.split() if w not in STOPWORDS]
    tokens = [LEMMATIZER.lemmatize(w) for w in tokens]
    return " ".join(tokens)

# Test
print(generic_preprocess(df['review'].iloc[0]))

**Preprocessing**

In [ ]:
import unicodedata


# normalize and remove non-letter characters (keeps spaces)

In [ ]:
def strip_non_letters(text):

    norm = unicodedata.normalize('NFKD', str(text))
    return re.sub(r'[^a-zA-Z\s]', ' ', norm)

 # remove emojis / special by keeping only letters and spaces

In [ ]:
def imdb_preprocess(text):
    t = str(text).lower()
    t = URL_RE.sub(' ', t)
    t = HTML_RE.sub(' ', t)
    t = strip_non_letters(t)
    t = re.sub(r'\s+', ' ', t).strip()
    tokens = [w for w in t.split() if w not in STOPWORDS]
    tokens = [LEMMATIZER.lemmatize(w) for w in tokens]
    return " ".join(tokens)

# Apply both pipelines and store columns

In [ ]:
df['clean_generic'] = df['review'].apply(generic_preprocess)
df['clean_imdb'] = df['review'].apply(imdb_preprocess)

# map sentiments to numbers

In [ ]:
df['label'] = df['sentiment'].map({'positive':1,'negative':0})

# show examples

In [ ]:
df[['review','clean_generic','clean_imdb','label']].head(6)

youssef tarek abdelmonem

In [ ]:
orig_tokens = df['review'].astype(str).str.split().str.len()
gen_tokens = df['clean_generic'].astype(str).str.split().str.len()
imdb_tokens = df['clean_imdb'].astype(str).str.split().str.len()


In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(orig_tokens, bins=80, label='original', alpha=0.5)
sns.histplot(gen_tokens, bins=80, label='clean_generic', alpha=0.5)
sns.histplot(imdb_tokens, bins=80, label='clean_imdb', alpha=0.5)
plt.legend()
plt.title("Token length: before vs generic vs imdb-clean")
plt.show()


In [ ]:
from wordcloud import WordCloud
pos_text = " ".join(df[df['label']==1]['clean_imdb'].values)
neg_text = " ".join(df[df['label']==0]['clean_imdb'].values)


In [ ]:
wc_pos = WordCloud(width=800,height=400,background_color='white').generate(pos_text)
wc_neg = WordCloud(width=800,height=400,background_color='white').generate(neg_text)
plt.figure(figsize=(12,5))
plt.imshow(wc_pos); plt.axis('off'); plt.title("Positive Reviews Words")
plt.show()
plt.figure(figsize=(12,5))
plt.imshow(wc_neg); plt.axis('off'); plt.title("Negative Reviews Words")
plt.show()


*** Hazem Mohamed Mahmoud Hassan **

In [ ]:
# Import required libraries and prepare the dataset

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
X = df['clean_imdb'].values
y = df['label'].values

In [ ]:
#  Split dataset into Train and Test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

In [ ]:
#  Vectorize text using TF-IDF


tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1,2))

X_train_t = tfidf.fit_transform(X_train)
X_test_t  = tfidf.transform(X_test)

print("Train TF-IDF shape:", X_train_t.shape)
print("Test  TF-IDF shape:", X_test_t.shape)

In [ ]:
# Train Logistic Regression model


logreg = LogisticRegression(max_iter=400)
logreg.fit(X_train_t, y_train)

print("Logistic Regression training done.")

In [ ]:
#  Evaluate the model performance

pred = logreg.predict(X_test_t)

print("Accuracy (TF-IDF + LR):", accuracy_score(y_test, pred))
print(classification_report(y_test, pred, target_names=['neg','pos']))

In [ ]:
# Confusion Matrix and Heatmap visualization

import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, pred)

plt.figure(figsize=(6,5))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['neg','pos'],
    yticklabels=['neg','pos']
)

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix — TF-IDF + Logistic Regression')
plt.show()


**Yousef** **Mohamed** **Abdeltawab** **Mohamed**

In [ ]:
# Save the Vectorizer (TF-IDF) and Logistic Regression Model

import joblib
import os

In [ ]:
os.makedirs("artifacts", exist_ok=True)
joblib.dump(tfidf, "artifacts/tfidf.joblib")
joblib.dump(logreg, "artifacts/logreg.joblib")
print(" tfidf.joblib و logreg.joblib artifacts/")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split

In [ ]:
# Hyperparameters
VOCAB_SIZE = 20000
SEQ_LEN    = 200    # Pad/Truncate each sentence to 200 tokens
EMBED_DIM  = 100
BATCH      = 128
EPOCHS     = 5

In [ ]:
# Data Preparation
texts  = df["clean_imdb"].astype(str).values
labels = df["label"].values

In [ ]:
# Train/Test Split
X_tr, X_te, y_tr, y_te = train_test_split(
    texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

In [ ]:
# Text Vectorization Layer
vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LEN
)

In [ ]:
# Fit vectorizer on training data
vectorize_layer.adapt(
    tf.data.Dataset.from_tensor_slices(X_tr).batch(256)
)

In [ ]:
# Dataset Pipeline
def make_dataset(X, y, batch=BATCH, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(10000, seed=42)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
# Train/Validation datasets
train_ds = make_dataset(X_tr, y_tr)
val_ds   = make_dataset(X_te, y_te, shuffle=False)

# Ahmed RAbie

# MODEL 1: Embedding -> Flatten -> Dense

In [ ]:
model = models.Sequential([
    layers.Embedding(VOCAB_SIZE, EMBED_DIM, input_length=SEQ_LEN),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),

    layers.Dropout(0.4),

    layers.Dense(1, activation='sigmoid')
])

# Compile the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# callbacks

In [ ]:
cb = [
    callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),

    callbacks.ModelCheckpoint('artifacts/keras_imdb_best.h5', save_best_only=True)
]


# Train the model


In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cb)


# Lighter alternative (recommended for weaker devices)

In [ ]:
model2 = models.Sequential([
    layers.Embedding(VOCAB_SIZE, EMBED_DIM, input_length=SEQ_LEN),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])


# **Youssef Magdy Ahmed**

##Deep model evaluation + training


**Evaluate**


In [ ]:
loss, acc = model.evaluate(val_ds, verbose=0)
print("Test accuracy (NN):", acc)

#Predictions (probabilities)


In [ ]:
import numpy as np
y_probs = model.predict(val_ds)
y_pred = (y_probs.flatten() >= 0.5).astype(int)

# True labels in same order


In [ ]:
y_true = np.concatenate([y for _,y in val_ds], axis=0)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
print(classification_report(y_true, y_pred, target_names=['neg','pos']))
print("ROC AUC:", roc_auc_score(y_true, y_probs))

# Confusion matrix plot

In [ ]:
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['neg','pos'], yticklabels=['neg','pos'])
plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix — NN')
plt.show()

# plot history


In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss'); plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss')

In [ ]:
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train_acc'); plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend(); plt.title('Accuracy')
plt.show()